<a href="https://colab.research.google.com/github/barbarosugurkan/kahvelab-beam-analysis/blob/main/2025-07-20/2025-07-20_understanding_cage_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Function to extract ADC reading from a file
def extract_value_from_file(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            if "ADC : channel #   0 reading =" in line:
                value_str = line.split("ADC : channel #   0 reading =")[1].split("+-")[0].strip()
                try:
                    return float(value_str)
                except ValueError:
                    print(f"⚠️ Error converting value in {file_path}: Not a valid float.")
                    return None
    print(f"⚠️ Warning: No ADC reading found in {file_path}.")
    return None

# Burada yaptığımız mesela 1 isimli klasöre girip orada işlem yapmak
# sonuç olarak yaptığımız işlemde tek tek o klasör içindeki dosyaları açıp
# non-zero sayıları row_values şeklinde bir listeye atmak.
def process_files_in_subdirectory(subdirectory_path, row_index):
    row_values = []
    file_order = []  # To store filenames for matrix verification

    # Sorting ADC files numerically, handling decimal numbers correctly
    files = sorted(
        [f for f in os.listdir(subdirectory_path) if f.endswith(".adc")],
        key=lambda x: float(x.split('.')[0].replace('SEMtest', ''))  # Ensures correct sorting of decimal values
    )

    # eğer SEMtest0.0.adc, SEMtest0.1.adc şeklinde dosyalarımız varsa files listesinde bunların sıralı bir şekilde isimleri olacak

    print(f"📂 Sorted files in {subdirectory_path}: {files}")  # Debugging: Check correct order

    for col_index, filename in enumerate(files): #enumarate bize index ve liste elemanını veriyor
        #col_index 0'dan başlayan indexler, filename de dosya adları
        #col_index dediğimiz dosya numarasıyla ilişkili bir şey
        file_path = os.path.join(subdirectory_path, filename)
        value = extract_value_from_file(file_path) #non-zero sayısını çıkardık

        row_values.append(value if value is not None else np.nan)  # Fill missing values with NaN
        file_order.append(filename)  # Track file mapping

        # Print matrix assignment info
        print(f"📝 Matrix[{row_index}, {col_index}] ← {filename}")

    # eğer 20'den az değer varsa kalanları np.nan ile doldur
    while len(row_values) < 20:
        row_values.append(np.nan)  # Fill missing entries
        file_order.append("MISSING")  # Track missing values

    return row_values[:20], file_order[:20]  # Enforce only 20 values per row

# Main directory dediğimiz 1 2 3 gibi klasörleri içeren ana klasör
def process_rows_in_directory(main_directory_path):
    all_rows_values = []
    matrix_file_order = []  # Store file names for verification

    # Sorting the folders numerically from 1 to 20
    folders = sorted(
        [f for f in os.listdir(main_directory_path) if f.isdigit()],
        key=lambda x: int(x)  # Ensures correct numerical sorting
    )

    print(f"📁 Sorted folders in {main_directory_path}: {folders}")  # Debugging: Check correct order

    for row_index, row_folder in enumerate(folders):
        subdirectory_path = os.path.join(main_directory_path, row_folder)
        if os.path.isdir(subdirectory_path):
            row_values, file_order = process_files_in_subdirectory(subdirectory_path, row_index)
            all_rows_values.append(row_values)
            matrix_file_order.append(file_order)

        # all_rows_values dediği aslında 1 2 3 4 gibi klasör kaç tane varsa o kadar
        # liste içeren bir liste bizim durumumuzda bunun sayısı anladığım 20 yani 20x20lik bir matrix elde edebiliriz

    return all_rows_values, matrix_file_order

# Define main directory path (UPDATE this path before running)
main_directory_path = "/content/drive/MyDrive/hezin.cage"

# Process data
Average_values_of_100, matrix_file_order = process_rows_in_directory(main_directory_path)

# Convert the extracted data into a NumPy array
matrix_data = np.array(Average_values_of_100)  # This is now 20x20
matrix_file_order_np = np.array(matrix_file_order)  # Store file names

# Verify matrix dimensions
print(f"✅ Matrix shape: {matrix_data.shape}")  # Should be (20, 20)

# Print matrix file mapping
print("\n📌 File Mapping in Matrix (Folder/File Order):\n")
for row in matrix_file_order_np:
    print(row)

# Function to Create a Plotly Visualization
def create_plotly_visualization(matrix_data, title):
    N, M = matrix_data.shape  # Get matrix dimensions
    x_coords, y_coords = np.meshgrid(np.arange(M) * 0.5, np.arange(N) * 0.5)  # Generate coordinates

    # Flatten arrays for plotting
    x_flat = x_coords.flatten()
    y_flat = y_coords.flatten()
    quantities_flat = matrix_data.flatten()

    # Create an interactive scatter plot
    fig = go.Figure(data=go.Scatter(
        x=x_flat,
        y=y_flat,
        mode='markers',
        marker=dict(
            size=10,  # Marker size
            color=quantities_flat,  # ADC values define color
            colorscale='gray_r',  # Grayscale colormap
            showscale=True,  # Display color bar
            colorbar=dict(title="ADC Value"),
        )
    ))

    # Update layout settings
    fig.update_layout(
        title=title,
        xaxis_title="X Axis (mm)",
        yaxis_title="Y Axis (mm)",
        yaxis_autorange='reversed',  # Keep Y-axis aligned correctly
        width=800,
        height=800,
    )

    fig.show()

# Create and display the visualization
create_plotly_visualization(matrix_data, "Cage250V")


📁 Sorted folders in /content/drive/MyDrive/hezin.cage: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
📂 Sorted files in /content/drive/MyDrive/hezin.cage/1: ['SEMtest0.5.adc', 'SEMtest0.2.adc', 'SEMtest0.6.adc', 'SEMtest0.7.adc', 'SEMtest0.3.adc', 'SEMtest0.9.adc', 'SEMtest0.8.adc', 'SEMtest0.1.adc', 'SEMtest0.0.adc', 'SEMtest0.4.adc', 'SEMtest1.3.adc', 'SEMtest1.2.adc', 'SEMtest1.0.adc', 'SEMtest1.9.adc', 'SEMtest1.8.adc', 'SEMtest1.6.adc', 'SEMtest1.7.adc', 'SEMtest1.4.adc', 'SEMtest1.5.adc', 'SEMtest1.1.adc', 'SEMtest2.0.adc']
📝 Matrix[0, 0] ← SEMtest0.5.adc
📝 Matrix[0, 1] ← SEMtest0.2.adc
📝 Matrix[0, 2] ← SEMtest0.6.adc
📝 Matrix[0, 3] ← SEMtest0.7.adc
📝 Matrix[0, 4] ← SEMtest0.3.adc
📝 Matrix[0, 5] ← SEMtest0.9.adc
📝 Matrix[0, 6] ← SEMtest0.8.adc
📝 Matrix[0, 7] ← SEMtest0.1.adc
📝 Matrix[0, 8] ← SEMtest0.0.adc
📝 Matrix[0, 9] ← SEMtest0.4.adc
📝 Matrix[0, 10] ← SEMtest1.3.adc
📝 Matrix[0, 11] ← SEMtest1.2.adc
📝 Matrix[0, 1

# **vertical tarama kodu**

In [ ]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Regular expression to extract numeric values from filenames like "SEMtest0.5.adc"
def extract_number(filename):
    match = re.search(r"SEMtest(\d+\.\d+|\d+)", filename)
    return float(match.group(1)) if match else float('inf')  # Default to inf if no match found

# Function to extract ADC reading from a file
def extract_value_from_file(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            if "ADC : channel #   0 reading =" in line:
                value_str = line.split("ADC : channel #   0 reading =")[1].split("+-")[0].strip()
                try:
                    return float(value_str)
                except ValueError:
                    print(f"⚠️ Error converting value in {file_path}: Not a valid float.")
                    return None
    print(f"⚠️ Warning: No ADC reading found in {file_path}.")
    return None

# Function to process all .adc files in a given subdirectory
def process_files_in_subdirectory(subdirectory_path, column_index):
    column_values = []
    file_order = []  # Track filenames for debugging

    # Sorting files numerically, ensuring 0.0 to 2.0 order
    files = sorted(
        [f for f in os.listdir(subdirectory_path) if f.endswith(".adc")],
        key=extract_number  # Sort by extracted floating-point number
    )

    print(f"📂 Sorted files in {subdirectory_path}: {files}")  # Debugging: Verify order

    for row_index, filename in enumerate(files):
        file_path = os.path.join(subdirectory_path, filename)
        value = extract_value_from_file(file_path)

        column_values.append(value if value is not None else np.nan)  # Handle missing values
        file_order.append(filename)  # Track file mapping

        # Print matrix assignment info
        print(f"📝 Matrix[{column_index}, {row_index}] ← {filename}")

    # Ensure exactly 20 values per row
    while len(column_values) < 20:
        column_values.append(np.nan)  # Fill missing entries
        file_order.append("MISSING")

    return column_values[:20], file_order[:20]  # Return only 20 values

# Function to process all folders (columns) in the main directory
def process_columns_in_directory(main_directory_path):
    all_columns_values = []
    matrix_file_order = []  # Track file order for debugging

    # Sorting folders numerically (1 to 20)
    folders = sorted(
        [f for f in os.listdir(main_directory_path) if f.isdigit()],
        key=lambda x: int(x)  # Sort as integers
    )

    print(f"📁 Sorted folders in {main_directory_path}: {folders}")  # Debugging: Verify order

    for column_index, column_folder in enumerate(folders):
        subdirectory_path = os.path.join(main_directory_path, column_folder)
        if os.path.isdir(subdirectory_path):
            column_values, file_order = process_files_in_subdirectory(subdirectory_path, column_index)
            all_columns_values.append(column_values)
            matrix_file_order.append(file_order)

    return all_columns_values, matrix_file_order

# Define main directory path (UPDATE this path before running)
main_directory_path = "/content/drive/MyDrive/hezin.cage"

# Process data
Average_values_of_100, matrix_file_order = process_columns_in_directory(main_directory_path)

# Convert the extracted data into a NumPy array
matrix_data = np.array(Average_values_of_100).T  # Should be 20x20
matrix_file_order_np = np.array(matrix_file_order).T  # Store file names

# Verify matrix dimensions
print(f"✅ Matrix shape: {matrix_data.shape}")  # Should be (20, 20)

# Print matrix file mapping
print("\n📌 File Mapping in Matrix (Folder/File Order):\n")
for row in matrix_file_order_np:
    print(row)

# Function to Create a Plotly Visualization
def create_plotly_visualization(matrix_data, title):
    N, M = matrix_data.shape  # Get matrix dimensions
    x_coords, y_coords = np.meshgrid(np.arange(M) * 0.5, np.arange(N) * 0.5)  # Generate coordinates

    # Flatten arrays for plotting
    x_flat = x_coords.flatten()
    y_flat = y_coords.flatten()
    quantities_flat = matrix_data.flatten()

    # Create an interactive scatter plot
    fig = go.Figure(data=go.Scatter(
        x=x_flat,
        y=y_flat,
        mode='markers',
        marker=dict(
            size=10,  # Marker size
            color=quantities_flat,  # ADC values define color
            colorscale='gray_r',  # Grayscale colormap
            showscale=True,  # Display color bar
            colorbar=dict(title="ADC Value"),
        )
    ))

    # Update layout settings
    fig.update_layout(
        title=title,
        xaxis_title="X Axis (mm)",
        yaxis_title="Y Axis (mm)",
        yaxis_autorange='reversed',  # Keep Y-axis aligned correctly
        width=600,
        height=600,
    )

    fig.show()

# Create and display the visualization
create_plotly_visualization(matrix_data, "Cage250V")


📁 Sorted folders in /content/drive/MyDrive/hezin.cage: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
📂 Sorted files in /content/drive/MyDrive/hezin.cage/1: ['SEMtest0.0.adc', 'SEMtest0.1.adc', 'SEMtest0.2.adc', 'SEMtest0.3.adc', 'SEMtest0.4.adc', 'SEMtest0.5.adc', 'SEMtest0.6.adc', 'SEMtest0.7.adc', 'SEMtest0.8.adc', 'SEMtest0.9.adc', 'SEMtest1.0.adc', 'SEMtest1.1.adc', 'SEMtest1.2.adc', 'SEMtest1.3.adc', 'SEMtest1.4.adc', 'SEMtest1.5.adc', 'SEMtest1.6.adc', 'SEMtest1.7.adc', 'SEMtest1.8.adc', 'SEMtest1.9.adc', 'SEMtest2.0.adc']
📝 Matrix[0, 0] ← SEMtest0.0.adc
📝 Matrix[0, 1] ← SEMtest0.1.adc
📝 Matrix[0, 2] ← SEMtest0.2.adc
📝 Matrix[0, 3] ← SEMtest0.3.adc
📝 Matrix[0, 4] ← SEMtest0.4.adc
📝 Matrix[0, 5] ← SEMtest0.5.adc
📝 Matrix[0, 6] ← SEMtest0.6.adc
📝 Matrix[0, 7] ← SEMtest0.7.adc
📝 Matrix[0, 8] ← SEMtest0.8.adc
📝 Matrix[0, 9] ← SEMtest0.9.adc
📝 Matrix[0, 10] ← SEMtest1.0.adc
📝 Matrix[0, 11] ← SEMtest1.1.adc
📝 Matrix[0, 1